In [12]:
import rospy
import numpy as np
from matplotlib import pyplot
import math as m
import random
import pandas as pd



In [11]:
class params:
    # Constriction coefficients
    kappa = 1
    phi1 = 2.05
    phi2 = 2.05
    phi = phi1 + phi2
    chi = 1.37              # 2*kappa/abs(2-phi-m.sqrt(phi**2-4*phi))

    # other params
    max_iter = 2         # maximum number of iterations
    nPart = 5               # no of drones/particles
    w = chi                 # inertia coefficient
    wdamp = 1               # damping coefficent for inertia
    c1 = chi*phi1           # personal acceleration coefficient
    c2 = chi*phi2           # social acceleration coefficient
    showIterinfo = True     # Flag for showing iteration information

class particle(params):
    position = np.ndarray(shape=(params.nPart,3),dtype=np.uint32)                     
    velocity = np.ndarray(shape=(params.nPart,3),dtype=np.uint32)                   
    cost = np.ndarray(shape=(params.nPart,1),dtype=np.uint32)                  
    Best_cost = np.ndarray(shape=(params.nPart,1),dtype=np.uint32)          
    Best_position = np.ndarray(shape=(params.nPart,3),dtype=np.uint32)          
    Global_best_cost = m.inf
    Global_best_position = np.ndarray(shape=(params.nPart,3),dtype=np.uint32)

    df1=pd.DataFrame(position, columns=['x','y','z'])
    df2=pd.DataFrame(velocity, columns=['velx','vely','velz'])
    df3=pd.DataFrame(cost, columns=['Cost'])
    df4=pd.DataFrame(Best_cost, columns=['Best Cost'])
    df5=pd.DataFrame(Best_position, columns=['Best_x','Best_y','Best_z'])
    # df6=pd.DataFrame(Global_best_cost, columns='Global_Cost') 
    df7=pd.DataFrame(Global_best_position, columns=['Global_x','Global_y','Global_z'])

    data = pd.concat([df1, df2, df3, df4,df5,df7], axis=1)

In [19]:
class PSO(particle):
    def __init__(self):
        # rospy.init_node("pso_search", anonymous=True)
        # self.Rate = rospy.rate(10)
        self.nvar = 3
        self.varMin = 0
        self.varMax = 10
        self.goal = np.array([3,3,3])

    # Cost function to minimize the distance error 
    # between the goal and current position of each drone
    def costfn(self,position):
        cost = 0
        for i in range(len(position)):
            cost += np.subtract(self.goal[i],position[i])
            print(self.goal[i],position[i])

        return cost
    
    def initialization(self):
        for i in range(params.nPart):
            particle.position[i,:] = np.array([random.randint(self.varMin,self.varMax),random.randint(self.varMin,self.varMax),random.randint(self.varMin,self.varMax)])
            particle.cost[i] = self.costfn(particle.position[i,:])
            particle.Best_cost[i] = particle.cost[i]
            particle.Best_position[i,:] = particle.position[i,:]

            if particle.cost[i] < particle.Global_best_cost:
                particle.Global_best_cost = particle.cost[i]
                particle.Global_best_position[i,:] = particle.position[i,:]
                # print(particle.cost[i])
        
        self.Bestcosts = np.zeros(shape=(params.max_iter,1))

    def main(self):
        self.initialization()
        for j in range(params.max_iter):
            for i in range(params.nPart):
                particle.velocity[i,:] = params.w * particle.velocity[i,:] + params.c1 * random.randint(1,self.nvar) * np.subtract(particle.Best_position[i,:],particle.position[i,:]) + params.c2 * random.randint(1,self.nvar) * np.subtract(particle.Global_best_position[i,:],particle.position[i,:])

                particle.position[i,:] += particle.velocity[i,:]

                particle.cost[i] = self.costfn(particle.position[i,:])

                if particle.cost[i] < particle.Best_cost[i]:
                    particle.Best_cost[i] = particle.cost[i]
                    particle.Best_position[i,:] = particle.position[i,:]

                    if particle.Best_cost[i] < particle.Global_best_cost:
                        particle.Global_best_cost = particle.Best_cost[i]
                        particle.Global_best_position[i,:] = particle.position[i,:]
        
            self.Bestcosts[j] = particle.Global_best_cost

            if params.showIterinfo:
                # print("Iteration",str(j),": Best Cost",str(self.Bestcosts[j]))
                pass

            params.w *= params.wdamp

if __name__ == "__main__":
    optimize = PSO()
    optimize.main()

3 4
3 6
3 9
3 4
3 8
3 9
3 9
3 9
3 9
3 10
3 5
3 2
3 3
3 5
3 5


ValueError: could not broadcast input array from shape (3,) into shape (1,)

In [5]:

particle = particle()
df1=pd.DataFrame(particle.position)
df2=pd.DataFrame(particle.velocity)
df3=pd.DataFrame(particle.cost)
df4=pd.DataFrame(particle.Best_cost)
df5=pd.DataFrame(particle.Best_position)
#df6=pd.DataFrame(particle.Global_best_cost) 
df7=pd.DataFrame(particle.Global_best_position)

In [6]:
df1.columns=['x','y','z']
df2.columns=['v']
df3.columns=['cost']
df4.columns=['bestcost']
df5.columns=['posx','posy','posz']

In [10]:
merged_df = pd.concat([df1, df2, df3, df4], axis=1)
# merged_df.iloc[0:5,3:5]
df5['posz']

0    0
1    0
2    0
3    0
4    0
Name: posz, dtype: uint32

In [61]:
df

[   x  y  z
 0  0  0  0
 1  0  0  0
 2  0  0  0
 3  0  0  0
 4  0  0  0,
    v
 0  0
 1  0
 2  0
 3  0
 4  0,
    cost
 0     0
 1     0
 2     0
 3     0
 4     0,
    bestcost
 0         0
 1         0
 2         0
 3         0
 4         0]

In [64]:
import pandas as pd

# Sample data (assuming all DataFrames have the same number of rows)
df1 = pd.DataFrame({'x': [1, 2], 'y': [3, 4], 'z': [5, 6]})
df2 = pd.DataFrame({'v': [7, 8]})
df3 = pd.DataFrame({'cost': [9, 10]})
df4 = pd.DataFrame({'bestcost': [11, 12]})

# Concatenating the DataFrames
merged_df = pd.concat([df1, df2, df3, df4], axis=1)

# Creating a MultiIndex for columns
merged_df.columns = pd.MultiIndex.from_tuples([
    ('data', 'x'), ('data', 'y'), ('data', 'z'),
    ('metrics', 'v'), ('metrics', 'cost'), ('metrics', 'bestcost')
])

# Displaying the DataFrame with multi-level columns
print(merged_df)


  data       metrics              
     x  y  z       v cost bestcost
0    1  3  5       7    9       11
1    2  4  6       8   10       12
